# Tranform demos

This notebook demonstrates the transforms provided by torchvideo.

First we need to do some set up, and load some data to play with

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('../src')
print(sys.executable)
print(sys.version)

/usr/bin/python3
3.7.2 (default, Dec 29 2018, 21:15:15) 
[GCC 8.2.1 20181127]


In [ ]:
from torchvideo.transforms import *
from torchvideo.datasets import ImageFolderVideoDataset
from torchvideo.datasets.vis import show_video
from torchvision.transforms import Compose, Lambda, Grayscale

# original video size = 640x368

transform = Compose([
    CollectFrames(),
    PILVideoToTensor()
])

dataset = ImageFolderVideoDataset('../tests/data/media/video_image_folder/', 'frame_{:05d}.jpg', transform=Compose([]))
video = list(dataset[0])

In [ ]:
type(video), type(video[0])

In [ ]:
# Original
dataset.transform = Compose([
    CollectFrames(),
    PILVideoToTensor(),
])
show_video(dataset[0])

In [ ]:
from moviepy.editor import *
import torch
import numpy as np


def tensor_video2clip(tensor_video, fps=24):
    # CTHW -> THWC
    vid = tensor_video.mul_(255).to(torch.uint8).permute(1, 2, 3, 0).cpu().numpy()
    return ImageSequenceClip(list(vid), fps=fps)


def flat2grid(seq, n_cols):
    n_rows = int(np.ceil(len(seq) / n_cols))
    grid = []
    for irow in range(n_rows):
        row = []
        grid.append(row)
        for icol in range(n_cols):
            i = irow * n_cols + icol
            if i == len(seq):
                break
            row.append(seq[i])
    return grid
    

def demo_transform(transform, n_samples=1, tile_width=3, fps=24):
    if n_samples > tile_width and not (n_samples / tile_width).is_integer():
        raise ValueError("Expected tile_width to divide n_samples perfectly.")
    samples = []
    for i in range(n_samples):
        samples.append(tensor_video2clip(transform(video), fps=fps))
    samples = flat2grid(samples, tile_width)
        
    return clips_array(samples).ipython_display()

... and now we can play with transforms!

---

# [PIL Video transforms](https://torchvideo.readthedocs.io/en/latest/transforms.html#transforms-on-pil-videos)

## [CenterCropVideo](https://torchvideo.readthedocs.io/en/latest/transforms.html#centercropvideo)

In [ ]:
demo_transform(Compose([
    CenterCropVideo((200, 400)),
    CollectFrames(),
    PILVideoToTensor()
]))

## [RandomCropVideo](https://torchvideo.readthedocs.io/en/latest/transforms.html#randomcropvideo)

In [ ]:
demo_transform(Compose([
    RandomCropVideo((150, 300)),
    CollectFrames(),
    PILVideoToTensor()
]), n_samples=9)

In [ ]:
demo_transform(Compose([
    ResizeVideo((100, 200)),
    RandomCropVideo((150, 300), pad_if_needed=True),
    CollectFrames(),
    PILVideoToTensor()
]), n_samples=9)

In [ ]:
demo_transform(Compose([
    ResizeVideo((130, 280)),
    RandomCropVideo((150, 300), padding=(20, 20, 0, 0)),
    CollectFrames(),
    PILVideoToTensor()
]), n_samples=9)

## [RandomHorizontalFlipVideo](https://torchvideo.readthedocs.io/en/latest/transforms.html#randomhorizontalflipvideo)

In [ ]:
demo_transform(Compose([
    ResizeVideo((184, 320)),
    RandomHorizontalFlipVideo(),
    CollectFrames(),
    PILVideoToTensor()
]), n_samples=4, tile_width=2)

## [MultiScaleCropVideo](https://torchvideo.readthedocs.io/en/latest/transforms.html#multiscalecropvideo)

In [ ]:
demo_transform(Compose([
    ResizeVideo((184, 320)),
    MultiScaleCropVideo((100, 200), max_distortion=2),
    CollectFrames(),
    PILVideoToTensor(),
]), n_samples=9)

## [RandomResizedCropVideo](https://torchvideo.readthedocs.io/en/latest/transforms.html#randomresizedcropvideo)

In [ ]:
demo_transform(Compose([
    RandomResizedCropVideo((100, 200)),
    CollectFrames(),
    PILVideoToTensor(),
]), n_samples=9)

## [TimeApply](https://torchvideo.readthedocs.io/en/latest/transforms.html#timeapply)

In [ ]:
demo_transform(Compose([
    TimeApply(Grayscale()),
    CollectFrames(),
    PILVideoToTensor(),
]), n_samples=1)

---

# [Torch.*Tensor transforms](https://torchvideo.readthedocs.io/en/latest/transforms.html#transforms-on-torch-tensor-videos)

## [NormalizeVideo](https://torchvideo.readthedocs.io/en/latest/transforms.html#normalizevideo)

In [ ]:
dataset.transform = Compose([
    CollectFrames(),
    PILVideoToTensor(),
    NormalizeVideo(128, 100)
])
x = dataset[0].cpu().numpy()
x.shape, x.mean(), x.std()

In [ ]:
dataset.transform = Compose([
    CollectFrames(),
    Stack(),
    ToTorchFormatTensor(),
    GroupNormalize([128], [100])
])
x = dataset[0].cpu().numpy()
x.shape, x.mean(), x.std()

In [ ]:
torch.flip(dataset[0], (0,)).shape

## [TimeToChannel](https://torchvideo.readthedocs.io/en/latest/transforms.html#timetochannel)

In [ ]:
dataset.transform = Compose([
    CollectFrames(),
    PILVideoToTensor(),
    TimeToChannel()
])
dataset[0].cpu().shape